# **Pre-Process the Data**

Optimization 2b Notes: Tweak the model itself using different optimizer models.


*   Remove: Early Stopping, Drop Out and other suggested ChatGPT model tweaks from 1a and 1b.
* Test use of SGD Optimizer with Momentum vs. Adam
* Retain number of hidden layers at 8
*   Double # of neurons to 16 from original 8



In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
#from tensorflow.keras.callbacks import EarlyStopping

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
X=application_df.drop(columns=['EIN', 'NAME'], axis=1, inplace=True)
y=application_df['IS_SUCCESSFUL']
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [ ]:
# Generate categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [ ]:
# Check the number of unique values in each column
unique_counts = application_df[application_cat].nunique()
print("Unique value counts for each categorical column:")
print(unique_counts)

Unique value counts for each categorical column:
APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64


In [ ]:
# Identify columns with more than 10 unique values
columns_more_than_10 = unique_counts[unique_counts > 10].index.tolist()
print("\nColumns with more than 10 unique values:")
print(columns_more_than_10)


Columns with more than 10 unique values:
['APPLICATION_TYPE', 'CLASSIFICATION']


In [ ]:
# Determine the number of data points for each unique value in these columns
for column in columns_more_than_10:
    value_counts = application_df[column].value_counts()
    print(f"\nValue counts for column '{column}':")
    print(value_counts)


Value counts for column 'APPLICATION_TYPE':
APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

Value counts for column 'CLASSIFICATION':
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [ ]:
# Identify columns with fewer than 10 unique values (excluding CLASSIFICATION and APPLICATION_TYPE)
columns_to_group = unique_counts[(unique_counts < 10) & (unique_counts.index != 'CLASSIFICATION') & (unique_counts.index != 'APPLICATION_TYPE')].index.tolist()
print("\nColumns to group infrequent values into 'Other':")
print(columns_to_group)


Columns to group infrequent values into 'Other':
['AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Set the threshold for excluding low count categories
threshold = 3000
# Generate our categorical variable list
application_types_to_replace = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
unique_counts = application_df[application_types_to_replace].nunique()

# Identify columns with more than 10 unique values
columns_more_than_10 = unique_counts[unique_counts > 10].index.tolist()

# Loop through each column with more than 10 unique values
for column in columns_more_than_10:
    # Determine the number of data points for each unique value in the column
    value_counts = application_df[column].value_counts()
    print(f"\nValue counts for column '{column}':")
    print(value_counts)

    # Identify categories to exclude
    low_count_values = value_counts[value_counts < threshold].index
    print(f"Categories with counts below {threshold} in column '{column}':")
    print(low_count_values)

    # Replace low count categories with 'Other'
    application_df[column] = application_df[column].replace(low_count_values, 'Other')


Value counts for column 'APPLICATION_TYPE':
APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64
Categories with counts below 3000 in column 'APPLICATION_TYPE':
Index(['T4', 'T6', 'T5', 'T19', 'T8', 'T7', 'T10', 'T9', 'T13', 'T12', 'T2',
       'T25', 'T14', 'T29', 'T15', 'T17'],
      dtype='object', name='APPLICATION_TYPE')

Value counts for column 'CLASSIFICATION':
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64
Categories with counts below 3000 in column 'CLASSIFICATION':
Index(['C3000', 'C2100', 'C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700',
       'C2800', 'C7100', 'C1300', 'C1280', 'C1230', 'C1400

In [ ]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

In [ ]:
# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     7262
Name: count, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Set the threshold for excluding low count categories
threshold = 1000
# Generate our categorical variable list
classifications_to_replace = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
unique_classification_counts = application_df[classifications_to_replace].nunique()

# Identify columns with more than 10 unique values
columns_more_than_10 = unique_counts[unique_counts > 10].index.tolist()

# Loop through each column with more than 10 unique values
for column in columns_more_than_10:
    # Determine the number of data points for each unique value in the column
    value_classification_counts = application_df[column].value_counts()
    print(f"\nValue counts for column '{column}':")
    print(value_classification_counts)

    # Identify categories to exclude
    low_count_classification_values = value_classification_counts[value_classification_counts < threshold].index
    print(f"Categories with counts below {threshold} in column '{column}':")
    print(low_count_classification_values)

    # Replace low count categories with 'Other'
    application_df[column] = application_df[column].replace(low_count_classification_values, 'Other')


Value counts for column 'APPLICATION_TYPE':
APPLICATION_TYPE
T3       27037
Other     7262
Name: count, dtype: int64
Categories with counts below 1000 in column 'APPLICATION_TYPE':
Index([], dtype='object', name='APPLICATION_TYPE')

Value counts for column 'CLASSIFICATION':
CLASSIFICATION
C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: count, dtype: int64
Categories with counts below 1000 in column 'CLASSIFICATION':
Index([], dtype='object', name='CLASSIFICATION')


In [ ]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: count, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
df_dummies = pd.get_dummies(application_df)
print("\nDataFrame after applying pd.get_dummies:")
print(df_dummies)


DataFrame after applying pd.get_dummies:
       STATUS   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0           1      5000              1                    True   
1           1    108590              1                   False   
2           1      5000              0                    True   
3           1      6692              1                   False   
4           1    142590              1                   False   
...       ...       ...            ...                     ...   
34294       1      5000              0                    True   
34295       1      5000              0                    True   
34296       1      5000              0                   False   
34297       1      5000              1                    True   
34298       1  36500179              0                   False   

       APPLICATION_TYPE_T3  AFFILIATION_CompanySponsored  \
0                    False                         False   
1                     True                   

In [ ]:
# Convert boolean columns to integers
df_dummies = df_dummies.astype(int)

print("\nDataFrame after applying pd.get_dummies and converting to integers:")
print(df_dummies.head())


DataFrame after applying pd.get_dummies and converting to integers:
   STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                       1   
1       1   108590              1                       0   
2       1     5000              0                       1   
3       1     6692              1                       0   
4       1   142590              1                       0   

   APPLICATION_TYPE_T3  AFFILIATION_CompanySponsored  \
0                    0                             0   
1                    1                             0   
2                    0                             1   
3                    1                             1   
4                    1                             0   

   AFFILIATION_Family/Parent  AFFILIATION_Independent  AFFILIATION_National  \
0                          0                        1                     0   
1                          0                        1                

In [ ]:
print("\nData types of the DataFrame columns:")
print(df_dummies.dtypes)


Data types of the DataFrame columns:
STATUS                          int64
ASK_AMT                         int64
IS_SUCCESSFUL                   int64
APPLICATION_TYPE_Other          int64
APPLICATION_TYPE_T3             int64
AFFILIATION_CompanySponsored    int64
AFFILIATION_Family/Parent       int64
AFFILIATION_Independent         int64
AFFILIATION_National            int64
AFFILIATION_Other               int64
AFFILIATION_Regional            int64
CLASSIFICATION_C1000            int64
CLASSIFICATION_C1200            int64
CLASSIFICATION_C2000            int64
CLASSIFICATION_Other            int64
USE_CASE_CommunityServ          int64
USE_CASE_Heathcare              int64
USE_CASE_Other                  int64
USE_CASE_Preservation           int64
USE_CASE_ProductDev             int64
ORGANIZATION_Association        int64
ORGANIZATION_Co-operative       int64
ORGANIZATION_Corporation        int64
ORGANIZATION_Trust              int64
INCOME_AMT_0                    int64
INCOME_AMT_1

In [ ]:
# Split our preprocessed data into our features and target arrays
# Split the preprocessed data into a training and testing dataset

# Identify the target variable
target = 'IS_SUCCESSFUL'

# Identify the feature variables related to 'Classification' and 'Application_Type'
# Assuming 'CLASSIFICATION' and 'APPLICATION_TYPE' were the original columns
classification_columns = [col for col in df_dummies.columns if 'CLASSIFICATION' in col]
application_type_columns = [col for col in df_dummies.columns if 'APPLICATION_TYPE' in col]

# Combine the feature columns
feature_columns = classification_columns + application_type_columns

# Create the features (X) and target (y) arrays
X = df_dummies[feature_columns].values
y = df_dummies[target].values

print("Feature columns:", feature_columns)
print("\nFeatures (X):")
print(X)
print("\nTarget (y):")
print(y)

Feature columns: ['CLASSIFICATION_C1000', 'CLASSIFICATION_C1200', 'CLASSIFICATION_C2000', 'CLASSIFICATION_Other', 'APPLICATION_TYPE_Other', 'APPLICATION_TYPE_T3']

Features (X):
[[1 0 0 0 1 0]
 [0 0 1 0 0 1]
 [0 0 0 1 1 0]
 ...
 [0 0 1 0 0 1]
 [0 0 0 1 1 0]
 [1 0 0 0 0 1]]

Target (y):
[1 1 0 ... 0 1 0]


In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# **Compile, Train & Evaluate the Model**

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_feat_num=X_train_scaled.shape[1]

# Compile the Sequential model together and customize metrics
nn = tf.keras.models.Sequential()

#Define the model shape via setting the Input layer
nn.add(tf.keras.layers.Input(shape=(input_feat_num,)))

# First hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Sixth hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Seventh hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Eighth hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid")
)

# Compile the model with a different optimizer and learning rate
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
#nn.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Early stopping callback
#early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with validation split and early stopping
#fit_model = nn.fit(X_train_scaled, y_train, epochs=200, validation_split=0.2, batch_size=16, callbacks=[early_stopping])

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 16)                112       
                                                                 
 dense_36 (Dense)            (None, 16)                272       
                                                                 
 dense_37 (Dense)            (None, 16)                272       
                                                                 
 dense_38 (Dense)            (None, 16)                272       
                                                                 
 dense_39 (Dense)            (None, 16)                272       
                                                                 
 dense_40 (Dense)            (None, 16)                272       
                                                                 
 dense_41 (Dense)            (None, 16)               

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="SGD", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=25)

Epoch 1/25
804/804 [==============================] - 3s 3ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 2/25
804/804 [==============================] - 2s 3ms/step - loss: 0.6905 - accuracy: 0.5321
Epoch 3/25
804/804 [==============================] - 3s 4ms/step - loss: 0.6900 - accuracy: 0.5321
Epoch 4/25
804/804 [==============================] - 3s 3ms/step - loss: 0.6894 - accuracy: 0.5321
Epoch 5/25
804/804 [==============================] - 2s 3ms/step - loss: 0.6888 - accuracy: 0.5321
Epoch 6/25
804/804 [==============================] - 2s 3ms/step - loss: 0.6882 - accuracy: 0.5359
Epoch 7/25
804/804 [==============================] - 2s 3ms/step - loss: 0.6877 - accuracy: 0.5409
Epoch 8/25
804/804 [==============================] - 2s 3ms/step - loss: 0.6872 - accuracy: 0.5453
Epoch 9/25
804/804 [==============================] - 3s 4ms/step - loss: 0.6870 - accuracy: 0.5441
Epoch 10/25
804/804 [==============================] - 2s 3ms/step - loss: 0.6868 - accuracy: 0.5423

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6861 - accuracy: 0.5417 - 538ms/epoch - 2ms/step
Loss: 0.6861281991004944, Accuracy: 0.5416909456253052


In [ ]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')

print("Model saved to 'AlphabetSoupCharity.h5'")

Model saved to 'AlphabetSoupCharity.h5'


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
